In [ ]:
import numpy
import pandas
import datetime
import time

import nbclient as viz

To make a line-chart with static data:
<pre>
viz.static( source, type='area', **kwargs )
</pre>
First argument is data in JSON format

Here we use a simple convention of labeling variables:
* x: independent variable ( X-axis )
* y: response variable ( Y-axis )

For type 'line' X and Y are required arguments.

Y argument can take a list as value. X, Y could be numeric or categorical, as well as ordinal or date-time type of variables. ( Though, time-series make most sense of it. ) If Y parameter contains the list, Z parameter is derived from that list.

Other accepted optional arguments:
* title
* colormap
* xformat, yformat, zformat: custom tick / value formatting
* xlabel, ylabel, zlabel: labels for axes and legend
* xlog, ylog: logarithmic scale axis
* xlim, ylim: clip display boundaries

The latter might be better achieved with zoom setting bookmark. ( See examples below. )

Custom colormap:
* can be passed as a list of colors
* can be passed as a dictionary

Lets see some examples:

In [ ]:
df = pandas.DataFrame(numpy.random.randn(100, 3), columns=list('ABC'))
df['B'] = df['B'].apply(lambda b: 1 + 1000*abs(b))
df['C'] = df['C'].apply(lambda c: ['red','blue','yellow','green','purple','orange','lime'][int(abs(round(c)))])
df['D'] = df.index.map(lambda i: datetime.datetime.fromtimestamp(time.time() - 43*i).strftime('%Y-%m-%d %H:%M:%S'))
df['E'] = df['A'].apply(lambda a: 1000*a*a) - df['B']
df.head()

In [ ]:
data = df.to_json(orient='records')

In [ ]:
viz.static(data, type='area',
         x='D',
         y='B',
         title='Simple Area-Chart Example',
         interpolate=True, ### defaults to False
         mode='wiggle', ### defaults to 'base'
         height=400, width='100%')

Click on the plot will toggle the pin ( fixed x value selection ) which can be used with bookmark or png-napshot.

In [ ]:
df = pandas.read_csv('app/static/data/timeseries.csv')
df.head()

In [ ]:
data = df.to_json(orient='records')

If Y argument contains the list, Z is derived from that list, Z argument if passed will be ignored.

In [ ]:
viz.static(data, type='area',
         x='date',
         y=['New York','San Francisco','Austin'],
         xlabel='Date',
         ylabel='Temperature (F)',
         zlabel='City',
         xformat='%Y %b %d',
         colormap={'New York':'teal','San Francisco':'purple','Austin':'orange'},
         title='Line-Chart Example',
         ylim = [15,95],
         #xlim=['2012-01-01','2012-06-01'],
         height=500, width='100%')

In [ ]:
from IPython.display import IFrame

IFrame('%s/static/dabbc2f251936dad341db2c6f3457d29/mnormalizep320p708400s4039005190386746s110s975500t210322424122916356599726977316844x13406468733881348454736591y01' % viz.host(),
        height=400, width='100%')

To make a line-chart with dynamic data source:
<pre>
viz.dynamic( source, type='line', **kwargs )
</pre>
First argument is URL end-point to retrieve the data in JSON format. This will always show current data when accessed. With refresh = N argument passed in it will refresh display in N seconds. ( No persistent connection. )

In [ ]:
viz.dynamic('%s/sample' % viz.host(), type='area',
         x='B',
         y=['A','E'],
         colormap=['#06f','#f60'],
         xlim=[0,3000],
         ylim=[-6000,6000],
         title='Line-Chart Example ( Dynamic Data Source )',
         refresh=3, ### request for current data in 3 seconds ###
         mode='wiggle',
         interpolate=True,
         height=400, width='100%')

Finally, to make a line-chart streaming dynamic data source:
<pre>
viz.stream( channel, type='line', **kwargs )
</pre>
First argument is URL end-point emitting data. This will refresh display as new data arrived.
With argument stream = N passed in it will keep sliding time-window of size N seconds in display.
( Time-series type of data with persistent connection. )

Attention: X argument if passed will be ignored, as it assumed to be a timestamp.

In [ ]:
viz.stream('sample-io', type='area',
         y='B',
         title='Line-Chart Example ( Streaming Data )',
         ylim = [0,8000],
         xformat='%H:%M:%S',
         interpolate=True,
         mode='wiggle',
         window = 120, ### 2 minute sliding time window in display ###
         height=400, width='100%')